<a href="https://colab.research.google.com/github/larrygoyeau/bubble_segmentation_Mask_RCNN/blob/master/Comparative_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook, present a comparative analyses between Unet and Mask RCNN by computing the IOU and mAP scores and the time of execution on the test set.

In [1]:
#@title To load Unet, run this block.

%tensorflow_version 1.x
!git clone https://github.com/larrygoyeau/bubble_segmentation_Unet

# Install required libs
import os
os.system('pip install albumentations==0.4.5')
os.system('pip install -U efficientnet==1.0.0')
os.system('pip install image-classifiers==1.0.0')
os.system('pip install -U segmentation-models==1.0.0')

import sys
import random
from google.colab import files
import cv2
import keras
from keras.models import model_from_json
import numpy as np
import matplotlib.pyplot as plt
import albumentations as A
import segmentation_models as sm
import resource
import time
import tensorflow as tf
from google_drive_downloader import GoogleDriveDownloader as gdd
    
# helper function for data visualization    
def denormalize(x):
    """Scale image to range 0..1 for correct plot"""
    x_max = np.percentile(x, 98)
    x_min = np.percentile(x, 2)    
    x = (x - x_min) / (x_max - x_min)
    x = x.clip(0, 1)
    return x
    

# classes for data loading and preprocessing
class Dataset:
    
    def __init__(
            self, 
            images_dir=None,
            preprocessing=None,
            augmentation=None,
    ):
        self.ids_image = os.listdir(images_dir)
        
        if images_dir!=None:
          self.images_fps = [os.path.join(images_dir, image_id) for image_id in self.ids_image]
        else:
          self.images_fps=None
   
        self.preprocessing = preprocessing

        self.augmentation=augmentation
    
    def __getitem__(self, i):
        
        # read data
        if self.images_fps!=None:
          image = cv2.imread(self.images_fps[i])
        else:
          image = self.image
        if len(image)>2**11:
          image=image[:2**11,:]
        if len(image[0])>2**11:
          image=image[:,:2**11]
        shape_image=image.shape
        p=255/(image.max()-image.min())
        image=(image-image.min())*p
        image= image.astype(np.uint8)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        if self.augmentation:
          I=len(image)
          J=len(image[0])
          sample = self.augmentation(I,J)(image=image)
          image = sample['image']
        
        # apply preprocessing
        if self.preprocessing:
            sample = self.preprocessing(image=image)
            image= sample['image']
            
        return image, shape_image
        
    def __len__(self):
        return len(self.ids_image)

def round_clip_0_1(x, **kwargs):
    return x.round().clip(0, 1)


def get_validation_augmentation(I,J):
    """Add paddings"""
    if I>384 or J>544:
      test_transform = [A.PadIfNeeded(2**(int(np.log(I-1)/np.log(2))+1), 2**(int(np.log(J-1)/np.log(2))+1), border_mode=0)]
    else:
      test_transform = [A.PadIfNeeded(384, 544, border_mode=0)]
    return A.Compose(test_transform)

def get_preprocessing(preprocessing_fn):
    """Construct preprocessing transform
    
    Args:
        preprocessing_fn (callbale): data normalization function 
            (can be specific for each pretrained neural network)
    Return:
        transform: albumentations.Compose
    
    """
    
    _transform = [
        A.Lambda(image=preprocessing_fn),
    ]
    return A.Compose(_transform)

BACKBONE = 'efficientnetb3'
preprocess_input = sm.get_preprocessing(BACKBONE)
LR = 0.0001

# define network parameters
n_classes =3 # case for binary and multiclass segmentation
activation = 'softmax'

# define optimizer
optim = keras.optimizers.Adam(LR)

gdd.download_file_from_google_drive(file_id='1-1ot0U_PFeSs9pdpIbd3RoTOrxrOzxnb',
                                    dest_path='/content/model_Unet.json',
                                    unzip=False)
json_file = open('/content/model_Unet.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model_Unet = model_from_json(loaded_model_json)

# load weights into new model, you can change the path if you don't use colab
MODEL_PATH = '/content/model_Unet.h5'
gdd.download_file_from_google_drive(file_id='1fGFApwvmk64gmpmyEm_GNYb-YEP3fSt4',
                                    dest_path=MODEL_PATH,
                                    unzip=False)

model_Unet.load_weights(MODEL_PATH)

# compile keras model with defined optimozer, loss and metrics
model_Unet.compile(optim,loss='categorical_crossentropy')


TensorFlow 1.x selected.
Cloning into 'bubble_segmentation_Unet'...
remote: Enumerating objects: 117, done.
remote: Counting objects: 100% (117/117), done.
remote: Compressing objects: 100% (117/117), done.
remote: Total 1051 (delta 66), reused 0 (delta 0), pack-reused 934
Receiving objects: 100% (1051/1051), 340.72 MiB | 12.33 MiB/s, done.
Resolving deltas: 100% (640/640), done.


Using TensorFlow backend.


Segmentation Models: using `keras` framework.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [2]:
#@title To load Mask RCNN, run this block. { run: "auto" }
%tensorflow_version 1.x

import os
os.system('git clone https://github.com/larrygoyeau/Mask_RCNN')
os.system('pip install -r /content/Mask_RCNN/requirements.txt')
os.system('git clone https://github.com/larrygoyeau/bubble_segmentation_Mask_RCNN')

import sys
sys.path.append('/content/Mask_RCNN')

import resource
import random
import numpy as np
import cv2
import matplotlib.pyplot as plt
from google.colab import files
from skimage.measure import find_contours
from matplotlib.patches import Polygon
import requests

# Root directory of the project
ROOT_DIR = os.path.abspath("/content")

# Import Mask RCNN
from mrcnn.config import Config
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn import utils

%matplotlib inline 

# To increas the recursion limite
resource.setrlimit(resource.RLIMIT_STACK, [0x100000000, resource.RLIM_INFINITY])
sys.setrecursionlimit(0x1000000)

def get_ax(rows=1, cols=1, size=8):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Change the default size attribute to control the size
    of rendered images
    """
    #fif, ax = plt.subplots(rows, cols, figsize=(size*1.5, size*rows))
    return plt.subplots(rows, cols, figsize=(size*1.5, size*rows))

class InferenceConfig(Config):
    """Configuration for training on the toy shapes dataset.
    Derives from the base Config class and overrides values specific
    to the toy shapes dataset.
    """
    # Give the configuration a recognizable name
    NAME = "shapes"

    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    IMAGE_RESIZE_MODE = "pad64"
    RPN_NMS_THRESHOLD = 0.55
    DETECTION_MAX_INSTANCES=350
    MAX_GT_INSTANCES=350
    DETECTION_MIN_CONFIDENCE=0

    # Number of classes (including background)
    NUM_CLASSES = 1 + 1  # background + 1 shape

    IMAGE_MIN_DIM = 512
    IMAGE_MAX_DIM = 512


    # Use smaller anchors because our image and objects are small
    RPN_ANCHOR_SCALES = (8, 16, 32, 64, 128)  # anchor side in pixels

inference_config = InferenceConfig()

# To increas the recursion limite
resource.setrlimit(resource.RLIMIT_STACK, [0x100000000, resource.RLIM_INFINITY])
sys.setrecursionlimit(0x1000000)

DATA_DIR = '/content/bubble_segmentation_Mask_RCNN/data_set'

x_test_dir = os.path.join(DATA_DIR, 'image_test')
y_test_dir = os.path.join(DATA_DIR, 'interior_mask_test')

#Some utility functions:

def color_bubble(mask_of_one_bubble,mask,i,j,I,J,color_of_bubble_done,color_of_bubbles_to_be_done):
  if mask[i,j]<color_of_bubbles_to_be_done:
    mask[i,j]=color_of_bubble_done
    mask_of_one_bubble[i,j]=1
    if 0<j:
      color_bubble(mask_of_one_bubble,mask,i,j-1,I,J,color_of_bubble_done,color_of_bubbles_to_be_done)
    if i<I-1:
      color_bubble(mask_of_one_bubble,mask,i+1,j,I,J,color_of_bubble_done,color_of_bubbles_to_be_done)
    if 0<i:
      color_bubble(mask_of_one_bubble,mask,i-1,j,I,J,color_of_bubble_done,color_of_bubbles_to_be_done)
    if j<J-1:
      color_bubble(mask_of_one_bubble,mask,i,j+1,I,J,color_of_bubble_done,color_of_bubbles_to_be_done)

class ShapesDataset(utils.Dataset):


    def load_shapes(self,images_dir, masks_dir):

        # Add classes
        self.add_class("shapes", 1, "bubble")

        # Add images
        self.ids_image = os.listdir(images_dir)
        
        self.images_fps = [os.path.join(images_dir, image_id) for image_id in self.ids_image]
        if masks_dir!=None:
          self.masks_fps = [os.path.join(masks_dir,'mask'+image_id[5:]) for image_id in self.ids_image]
        else:
          self.masks_fps=None

        count=len(self.ids_image)
        
        for i in range(count):
            self.add_image("shapes", image_id=i, path=self.images_fps[i],path_mask=self.masks_fps[i])

    def load_image(self, image_id):
        """Generate an image from the specs of the given image ID.
        This function loads the image from a file
        """
        info = self.image_info[image_id]
        image = cv2.imread(info['path'])
        if len(image)>2**11:
          image=image[:2**11,:]
        if len(image[0])>2**11:
          image=image[:,:2**11]
        shape_image=image.shape
        p=255/(image.max()-image.min())
        image=(image-image.min())*p
        image= image.astype(np.uint8)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        return image

    def load_mask(self, image_id):
        """The annotations are the same as those used for Unet 
        except that this function allows you to create, 
        from a mask containing all the bubbles,
         a set of mask, each containing a bubble so that the annotation is indeed of the type 
         'instance' and not 'semantic'.
        """
        info = self.image_info[image_id]
        mask = cv2.imread(info['path_mask'], 0)
        I=len(mask)
        J=len(mask[0])
        number_of_bubbles=0
        list_of_mask=[]
        for i in range(I):
          for j in range(J):
            if mask[i,j]<5:
              mask_of_one_bubble=np.zeros( mask.shape, dtype=np.int32 )
              color_bubble(mask_of_one_bubble,mask,i,j,I,J,255,5)
              list_of_mask=list_of_mask+[mask_of_one_bubble]
              number_of_bubbles=number_of_bubbles+1
        list_of_mask=np.random.permutation(list_of_mask)
        mask_of_all_bubbles = np.stack(list_of_mask, axis=-1).astype('int')
        class_ids = np.ones((number_of_bubbles))
        return mask_of_all_bubbles.astype(np.bool), class_ids.astype(np.int32)

def semantic_mask(instance_mask):
  N=len(instance_mask[0][0])
  I=len(instance_mask)
  J=len(instance_mask[0])
  mask=np.zeros(instance_mask[...,0].shape)
  for n in range(N):
    for i in range(I):
      for j in range(J):
        if instance_mask[i,j,n]:
          mask[i,j]=1
  return mask

# Test dataset
dataset_test = ShapesDataset()
dataset_test.load_shapes(x_test_dir, y_test_dir)
dataset_test.prepare()

# Download trained weights from Releases if needed

def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)

MODEL_PATH = '/content/model.h5'

if not os.path.exists(MODEL_PATH):
    file_id = '16m6o97REebp_C86IbjbaHitxUvjBjrBe'
    destination = MODEL_PATH
    download_file_from_google_drive(file_id, destination)
    print("Pretrained model downloaded!")


# Recreate the model in inference mode
model_Mask_RCNN = modellib.MaskRCNN(mode="inference", 
                          config=inference_config,
                          model_dir=ROOT_DIR)

# Load trained weights
model_Mask_RCNN.load_weights(MODEL_PATH, by_name=True)

Pretrained model downloaded!


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
box_ind is deprecated, use box_indices instead


Instructions for updating:
Use `tf.cast` instead.


To compute mAP the predicted mask must be of instance type. As Unet give a semantic mask, we will save it and then use the function load_mask to get intance masks prediction.

In [3]:
if not os.path.exists('/content/pr_dir'):
  base_dir = 'pr_dir'
  os.mkdir(base_dir)

test_dataset = Dataset(
    x_test_dir, 
    preprocessing=get_preprocessing(preprocess_input),
    augmentation=get_validation_augmentation
)

image_names=os.listdir(x_test_dir)
n =len(image_names)
average_iou_Unet=0

for k in range(n):
    image, shape_image= test_dataset[k]
    image = np.expand_dims(image, axis=0)
    pr_mask = model_Unet.predict(image).round()[0]

    image=image[0]
    image=image[int((len(image)-shape_image[0])/2):int((len(image)-shape_image[0])/2)+shape_image[0]]
    image=image[:,int((len(image[0])-shape_image[1])/2):int((len(image[0])-shape_image[1])/2)+shape_image[1]]
    pr_mask=pr_mask[int((len(pr_mask)-shape_image[0])/2):int((len(pr_mask)-shape_image[0])/2)+shape_image[0]]
    pr_mask=pr_mask[:,int((len(pr_mask[0])-shape_image[1])/2):int((len(pr_mask[0])-shape_image[1])/2)+shape_image[1]]

    I=len(pr_mask)
    J=len(pr_mask[0])
    for i in range(I):
      for j in range(J):
        if all(pr_mask[i,j]==[1,0,0]): # [1,0,0] mean pixel of bubble
          pr_mask[i,j]=0
        else:
          pr_mask[i,j]=255

    cv2.imwrite('/content/pr_dir/mask_'+image_names[k][-9:],pr_mask.astype(np.uint8))

    #Let compute the IOU score
    path=os.path.join(y_test_dir, 'mask'+image_names[k][5:])
    gt_mask = cv2.imread(path,0)
    pr_mask=np.delete(pr_mask, 0, 2)
    pr_mask=np.delete(pr_mask, 0, 2)
    pr_mask=np.squeeze(pr_mask)
    with tf.Session() as sess:
      ypredT = tf.constant(pr_mask/255)
      ytrueT = tf.constant(gt_mask/255)
      iou,conf_mat = tf.metrics.mean_iou(ytrueT, ypredT, num_classes=3)
      sess.run(tf.local_variables_initializer())
      sess.run([conf_mat])
      miou = sess.run([iou])
    average_iou_Unet=average_iou_Unet+miou[0]
    #print(miou[0])

# Test dataset
dataset_test_Unet = ShapesDataset()
dataset_test_Unet.load_shapes(x_test_dir, '/content/pr_dir')
dataset_test_Unet.prepare()


Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


We compute the mAP for Unet and print it with the IOU

In [4]:
# Compute VOC-Style mAP @ IoU=0.5

image_ids =dataset_test.image_ids
APs = []
for image_id in image_ids:
    # Load image and ground truth data
    image, image_meta, gt_class_id, gt_bbox, gt_mask =\
        modellib.load_image_gt(dataset_test, inference_config,
                               image_id, use_mini_mask=False)

    # Load predicted mask
    image, image_meta, pr_class_id, pr_bbox, pr_mask =\
        modellib.load_image_gt(dataset_test_Unet, inference_config,
                               image_id, use_mini_mask=False)

    # Compute AP
    AP, precisions, recalls, overlaps =\
        utils.compute_ap(gt_bbox, gt_class_id, gt_mask,
                         pr_bbox, pr_class_id, np.ones((len(pr_class_id))), pr_mask)
    APs.append(AP)
    
print('Unet IOU: ', round(average_iou_Unet/n,3)) #Was computed in the previous block
print("Unet mAP: ", round(np.mean(APs),3))

Unet IOU:  0.811
Unet mAP:  0.792


We compute the mAP and IOU for Mask RCNN

In [5]:
# Compute VOC-Style mAP @ IoU=0.5

image_ids =dataset_test.image_ids
APs = []
average_iou_Mask_RCNN=0
for image_id in image_ids:
    # Load image and ground truth data
    image, image_meta, gt_class_id, gt_bbox, gt_mask =\
        modellib.load_image_gt(dataset_test, inference_config,
                               image_id, use_mini_mask=False)
    # Run object detection
    results = model_Mask_RCNN.detect([image], verbose=0)
    r = results[0]
    # Compute AP
    AP, precisions, recalls, overlaps =\
        utils.compute_ap(gt_bbox, gt_class_id, gt_mask,
                         r["rois"], r["class_ids"], np.ones((len(r['class_ids']))), r['masks'])
    APs.append(AP)

    semantic_pr_mask=semantic_mask(r['masks'])
    semantic_gt_mask = semantic_mask(gt_mask)
    with tf.Session() as sess:
      ypredT = tf.constant(semantic_pr_mask)
      ytrueT = tf.constant(semantic_gt_mask)
      iou,conf_mat = tf.metrics.mean_iou(ytrueT, ypredT, num_classes=3)
      sess.run(tf.local_variables_initializer())
      sess.run([conf_mat])
      miou = sess.run([iou])
    average_iou_Mask_RCNN=average_iou_Mask_RCNN+miou[0]
    #print(miou)

print('Mask RCNN IOU: ', round(average_iou_Mask_RCNN/n,3))  
print("Mask RCNN mAP: ", round(np.mean(APs),3))

Mask RCNN IOU:  0.82
Mask RCNN mAP:  0.904


Now let compare the time of segmentation of Unet with Mask RCNN

In [6]:
tic=time.perf_counter()

for k in range(n):
    image, shape_image= test_dataset[k]
    image = np.expand_dims(image, axis=0)
    pr_mask = model_Unet.predict(image).round()[0]

tac=time.perf_counter()

print('In average Unet took '+str(round((tac-tic)/n,2))+'s per image')

In average Unet took 0.05s per image


In [7]:
tic=time.perf_counter()

for image_name in image_names:
  path=os.path.join(x_test_dir, image_name)
  image = cv2.imread(path)
  results = model_Mask_RCNN.detect([image], verbose=1)

tac=time.perf_counter()

print('\n In average Mask RCNN took '+str(round((tac-tic)/n,2))+'s per image')

Processing 1 images
image                    shape: (128, 128, 3)         min:   63.00000  max:  255.00000  uint8
molded_images            shape: (1, 512, 512, 3)      min:  -60.70000  max:  151.10000  float64
image_metas              shape: (1, 14)               min:    0.00000  max:  512.00000  float64
anchors                  shape: (1, 65472, 4)         min:   -0.17712  max:    1.05188  float32
Processing 1 images
image                    shape: (256, 256, 3)         min:   34.00000  max:  255.00000  uint8
molded_images            shape: (1, 512, 512, 3)      min:  -87.70000  max:  151.10000  float64
image_metas              shape: (1, 14)               min:    0.00000  max:  512.00000  float64
anchors                  shape: (1, 65472, 4)         min:   -0.17712  max:    1.05188  float32
Processing 1 images
image                    shape: (128, 256, 3)         min:   87.00000  max:  255.00000  uint8
molded_images            shape: (1, 512, 1024, 3)     min:  -36.70000  max:  151.1